In [1]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image
from sklearn.model_selection import train_test_split


In [ ]:
class AIVSHumanDataset(Dataset):
    def __init__(self, csv_file, root_dir='', transform=None):
        """
        Args:
            csv_file (str): Path to the CSV file with annotations.
            root_dir (str): Directory with all the images. If csv has full paths, root_dir can be ''.
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        self.data = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = os.path.join(self.root_dir, self.data.iloc[idx]['file_name'])
        image = Image.open(img_path).convert('RGB')
        label = int(self.data.iloc[idx]['label'])

        if self.transform:
            image = self.transform(image)

        return image, label


In [ ]:
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],   # Mean for ImageNet
                         [0.229, 0.224, 0.225])   # Std for ImageNet
])

val_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])


In [ ]:
# Adjust the path to your CSV file
csv_path = '/kaggle/input/ai-vs-human-generated-dataset/train.csv'

df = pd.read_csv(csv_path)
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])

train_df.to_csv('train_split.csv', index=False)
val_df.to_csv('val_split.csv', index=False)


In [ ]:
train_dataset = AIVSHumanDataset(
    csv_file='train_split.csv',
    root_dir='/kaggle/input/ai-vs-human-generated-dataset/',  # or your image root if needed
    transform=train_transforms
)

val_dataset = AIVSHumanDataset(
    csv_file='val_split.csv',
    root_dir='/kaggle/input/ai-vs-human-generated-dataset/',  # or your image root if needed
    transform=val_transforms
)


In [ ]:
batch_size = 96

train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=2
)

val_loader = DataLoader(
    val_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=2
)


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load a pretrained ResNeXt model
model = models.resnext50_32x4d(pretrained=True)  # or resnext101_32x8d

# Freeze early layers if you want (optional)
for param in model.parameters():
    param.requires_grad = True  # By default, we unfreeze everything for fine-tuning

# Replace the final classification layer
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)  # 2 classes

model = model.to(device)

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)


In [ ]:
def train_one_epoch(model, criterion, optimizer, dataloader, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in dataloader:
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward
        loss.backward()
        optimizer.step()

        # Statistics
        running_loss += loss.item() * inputs.size(0)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc

def validate_one_epoch(model, criterion, dataloader, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc


In [ ]:
num_epochs = 40 # adjust as needed

for epoch in range(num_epochs):
    train_loss, train_acc = train_one_epoch(model, criterion, optimizer, train_loader, device)
    val_loss, val_acc = validate_one_epoch(model, criterion, val_loader, device)

    print(f"Epoch [{epoch+1}/{num_epochs}] "
          f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f} "
          f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")


In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torchvision import transforms

# -----------------------------
# 1. Load test.csv
# -----------------------------
test_df = pd.read_csv('/kaggle/input/ai-vs-human-generated-dataset/test.csv')  


# -----------------------------
# 2. Define a test Dataset
# -----------------------------
class AIVSHumanTestDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        # Get the file_name from the row
        file_name = '/kaggle/input/ai-vs-human-generated-dataset/' + self.df.iloc[idx]['id']
        # Load image
        image = Image.open(file_name).convert('RGB')
        # Apply transforms (if any)
        if self.transform:
            image = self.transform(image)
        return image

# -----------------------------
# 3. Create test transforms
# -----------------------------
test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# -----------------------------
# 4. Instantiate the Dataset & DataLoader
# -----------------------------
test_dataset = AIVSHumanTestDataset(test_df, transform=test_transforms)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# -----------------------------
# 5. Load your trained model
# -----------------------------
# Example: 
#   model = YourResNeXtModel(...)
#   model.load_state_dict(torch.load('path/to/model.pth'))
#   model.eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.eval()

# -----------------------------
# 6. Generate predictions
# -----------------------------
all_preds = []
with torch.no_grad():
    for images in test_loader:
        images = images.to(device)
        outputs = model(images)  # shape [batch_size, 2] if using nn.CrossEntropyLoss
        _, predicted = torch.max(outputs, 1)
        all_preds.extend(predicted.cpu().numpy().tolist())

# -----------------------------
# 7. Create submission DataFrame
# -----------------------------
# "id" must match the id in test.csv
# "label" is your predicted class (0 or 1)
submission_df = pd.DataFrame({
    'id': test_df['id'],      # from test.csv
    'label': all_preds        # from the model predictions
})

# -----------------------------
# 8. Save submission (no index)
# -----------------------------
submission_df.to_csv('submissionds2.csv', index=False)

print("Submission file saved as submission.csv!")


In [ ]:
!rm -rf /kaggle/working/*

In [ ]:
!pip install autogluon

In [ ]:
torch.cuda.empty_cache()

In [ ]:
!pip install lion_pytorch

In [ ]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import OneCycleLR

# Dataset Classes
class AIVSHumanDataset(Dataset):
    def __init__(self, csv_file, root_dir='', transform=None):
        self.data = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = os.path.join(self.root_dir, self.data.iloc[idx]['file_name'])
        image = Image.open(img_path).convert('RGB')
        label = int(self.data.iloc[idx]['label'])

        if self.transform:
            image = self.transform(image)

        return image, label

class AIVSHumanTestDataset(Dataset):
    def __init__(self, df, root_dir='', transform=None):
        self.df = df
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        file_name = os.path.join(self.root_dir, self.df.iloc[idx]['id'])
        image = Image.open(file_name).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image

# Model Architecture
class GANDetectorNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.base = models.efficientnet_b3(pretrained=True)
        
        # Modify first layer
        original_layer = self.base.features[0][0]
        self.base.features[0][0] = nn.Conv2d(3, original_layer.out_channels, 
                                            kernel_size=3, stride=2, padding=1, bias=False)
        
        # Frequency analysis branch
        self.freq_branch = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=7, padding=3),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=5, padding=2),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d(1)
        )
        
        num_ftrs = self.base.classifier[1].in_features
        self.base.classifier = nn.Identity()
        
        self.classifier = nn.Sequential(
            nn.Linear(num_ftrs + 64, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(512, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 2)
        )

    def forward(self, x):
        features = self.base(x)
        freq_features = self.freq_branch(x)
        freq_features = freq_features.view(freq_features.size(0), -1)
        combined = torch.cat([features, freq_features], dim=1)
        return self.classifier(combined)

# Training function
def train_one_epoch(model, criterion, optimizer, scheduler, dataloader, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for inputs, labels in dataloader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        with torch.cuda.amp.autocast():
            outputs = model(inputs)
            ce_loss = criterion(outputs, labels)
            pt = torch.exp(-ce_loss)
            focal_loss = ((1 - pt) ** 2 * ce_loss).mean()
            
            if inputs.size(0) > 1:
                perturbed = inputs + 0.001 * torch.randn_like(inputs)
                perturbed_outputs = model(perturbed)
                consistency_loss = nn.MSELoss()(outputs, perturbed_outputs)
                loss = focal_loss + 0.1 * consistency_loss
            else:
                loss = focal_loss
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()
        
        running_loss += loss.item() * inputs.size(0)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)
    
    return running_loss / total, correct / total

# Validation function
def validate_one_epoch(model, criterion, dataloader, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            with torch.cuda.amp.autocast():
                outputs = model(inputs)
                loss = criterion(outputs, labels)
            
            running_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    
    return running_loss / total, correct / total

# Main execution
def main():
    # Data transforms
    train_transforms = transforms.Compose([
        transforms.Resize((384, 384)),
        transforms.RandomCrop(336),
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0, hue=0),
        transforms.ToTensor(),
        transforms.Lambda(lambda x: x + torch.randn_like(x) * 0.01),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

    val_transforms = transforms.Compose([
        transforms.Resize((336, 336)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

    # Data preparation
    csv_path = '/kaggle/input/ai-vs-human-generated-dataset/train.csv'
    df = pd.read_csv(csv_path)
    train_df, val_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])

    train_df.to_csv('train_split.csv', index=False)
    val_df.to_csv('val_split.csv', index=False)

    # Create datasets
    train_dataset = AIVSHumanDataset(
        csv_file='train_split.csv',
        root_dir='/kaggle/input/ai-vs-human-generated-dataset/',
        transform=train_transforms
    )

    val_dataset = AIVSHumanDataset(
        csv_file='val_split.csv',
        root_dir='/kaggle/input/ai-vs-human-generated-dataset/',
        transform=val_transforms
    )

    # Create dataloaders
    batch_size = 16
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

    # Model setup
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = GANDetectorNet().to(device)

    # Training setup
    criterion = nn.CrossEntropyLoss(label_smoothing=0.15)
    optimizer = optim.AdamW(model.parameters(), lr=1e-3, weight_decay=0.02)
    num_epochs = 10
    
    scheduler = OneCycleLR(
        optimizer,
        max_lr=1e-3,
        epochs=num_epochs,
        steps_per_epoch=len(train_loader),
        pct_start=0.2,
        div_factor=25
    )

    # Training loop
    best_val_acc = 0.0
    for epoch in range(num_epochs):
        train_loss, train_acc = train_one_epoch(model, criterion, optimizer, scheduler, train_loader, device)
        val_loss, val_acc = validate_one_epoch(model, criterion, val_loader, device)
        
        print(f"Epoch [{epoch+1}/{num_epochs}] "
              f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f} "
              f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")
        
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), 'best_model.pth')

    # Generate predictions
    test_df = pd.read_csv('/kaggle/input/ai-vs-human-generated-dataset/test.csv')
    test_dataset = AIVSHumanTestDataset(
        df=test_df,
        root_dir='/kaggle/input/ai-vs-human-generated-dataset/',
        transform=val_transforms
    )
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

    # Load best model
    model.load_state_dict(torch.load('best_model.pth'))
    model.eval()

    # Generate predictions
    all_preds = []
    with torch.no_grad():
        for images in test_loader:
            images = images.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            all_preds.extend(predicted.cpu().numpy().tolist())

    # Create submission file
    submission_df = pd.DataFrame({
        'id': test_df['id'],
        'label': all_preds
    })
    submission_df.to_csv('submission.csv', index=False)
    print("Submission file saved as submission.csv!")

if __name__ == "__main__":
    main()

In [2]:
import os
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image

# Dataset Class for Test Data
class AIVSHumanTestDataset(Dataset):
    def __init__(self, df, root_dir='', transform=None):
        self.df = df
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        file_name = os.path.join(self.root_dir, self.df.iloc[idx]['id'])
        image = Image.open(file_name).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image

# Model Architecture (needs to match the saved model architecture)
class GANDetectorNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.base = models.efficientnet_b3(pretrained=True)
        
        # Modify first layer
        original_layer = self.base.features[0][0]
        self.base.features[0][0] = nn.Conv2d(3, original_layer.out_channels, 
                                            kernel_size=3, stride=2, padding=1, bias=False)
        
        # Frequency analysis branch
        self.freq_branch = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=7, padding=3),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=5, padding=2),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d(1)
        )
        
        num_ftrs = self.base.classifier[1].in_features
        self.base.classifier = nn.Identity()
        
        self.classifier = nn.Sequential(
            nn.Linear(num_ftrs + 64, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(512, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 2)
        )

    def forward(self, x):
        features = self.base(x)
        freq_features = self.freq_branch(x)
        freq_features = freq_features.view(freq_features.size(0), -1)
        combined = torch.cat([features, freq_features], dim=1)
        return self.classifier(combined)

def generate_predictions():
    # Set device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # Data transform for validation/testing
    val_transforms = transforms.Compose([
        transforms.Resize((336, 336)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

    # Load test data
    test_df = pd.read_csv('/kaggle/input/ai-vs-human-generated-dataset/test.csv')
    test_dataset = AIVSHumanTestDataset(
        df=test_df,
        root_dir='/kaggle/input/ai-vs-human-generated-dataset/',
        transform=val_transforms
    )
    
    # Create test dataloader
    batch_size = 16
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

    # Initialize model
    model = GANDetectorNet().to(device)
    
    # Load saved model weights
    model.load_state_dict(torch.load('best_model.pth'))
    model.eval()

    # Generate predictions
    all_preds = []
    with torch.no_grad():
        for images in test_loader:
            images = images.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            all_preds.extend(predicted.cpu().numpy().tolist())

    # Create submission file
    submission_df = pd.DataFrame({
        'id': test_df['id'],
        'label': all_preds
    })
    submission_df.to_csv('submission3.csv', index=False)
    print("Submission file saved as submission.csv!")

if __name__ == "__main__":
    generate_predictions()

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B3_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
<ipython-input-2-0c3ce6afe926>:97: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models f

Submission file saved as submission.csv!


In [ ]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import OneCycleLR

# Dataset Classes
class AIVSHumanDataset(Dataset):
    def __init__(self, csv_file, root_dir='', transform=None):
        self.data = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = os.path.join(self.root_dir, self.data.iloc[idx]['file_name'])
        image = Image.open(img_path).convert('RGB')
        label = int(self.data.iloc[idx]['label'])

        if self.transform:
            image = self.transform(image)

        return image, label

class AIVSHumanTestDataset(Dataset):
    def __init__(self, df, root_dir='', transform=None):
        self.df = df
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        file_name = os.path.join(self.root_dir, self.df.iloc[idx]['id'])
        image = Image.open(file_name).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image

# Model Architecture
class GANDetectorNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.base = models.efficientnet_b3(pretrained=True)
        
        # Modify first layer
        original_layer = self.base.features[0][0]
        self.base.features[0][0] = nn.Conv2d(3, original_layer.out_channels, 
                                            kernel_size=3, stride=2, padding=1, bias=False)
        
        # Frequency analysis branch
        self.freq_branch = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=7, padding=3),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=5, padding=2),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d(1)
        )
        
        num_ftrs = self.base.classifier[1].in_features
        self.base.classifier = nn.Identity()
        
        self.classifier = nn.Sequential(
            nn.Linear(num_ftrs + 64, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(512, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 2)
        )

    def forward(self, x):
        features = self.base(x)
        freq_features = self.freq_branch(x)
        freq_features = freq_features.view(freq_features.size(0), -1)
        combined = torch.cat([features, freq_features], dim=1)
        return self.classifier(combined)

# Training function
def train_one_epoch(model, criterion, optimizer, scheduler, dataloader, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for inputs, labels in dataloader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        with torch.cuda.amp.autocast():
            outputs = model(inputs)
            ce_loss = criterion(outputs, labels)
            pt = torch.exp(-ce_loss)
            focal_loss = ((1 - pt) ** 2 * ce_loss).mean()
            
            if inputs.size(0) > 1:
                perturbed = inputs + 0.001 * torch.randn_like(inputs)
                perturbed_outputs = model(perturbed)
                consistency_loss = nn.MSELoss()(outputs, perturbed_outputs)
                loss = focal_loss + 0.1 * consistency_loss
            else:
                loss = focal_loss
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()
        
        running_loss += loss.item() * inputs.size(0)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)
    
    return running_loss / total, correct / total

# Validation function
def validate_one_epoch(model, criterion, dataloader, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            with torch.cuda.amp.autocast():
                outputs = model(inputs)
                loss = criterion(outputs, labels)
            
            running_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    
    return running_loss / total, correct / total



# [Previous code remains the same until the main() function]

def main():
    # Data transforms
    train_transforms = transforms.Compose([
        transforms.Resize((384, 384)),
        transforms.RandomCrop(336),
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0, hue=0),
        transforms.ToTensor(),
        transforms.Lambda(lambda x: x + torch.randn_like(x) * 0.01),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

    val_transforms = transforms.Compose([
        transforms.Resize((336, 336)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

    # Data preparation
    csv_path = '/kaggle/input/ai-vs-human-generated-dataset/train.csv'
    df = pd.read_csv(csv_path)
    train_df, val_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])

    train_df.to_csv('train_split.csv', index=False)
    val_df.to_csv('val_split.csv', index=False)

    # Create datasets
    train_dataset = AIVSHumanDataset(
        csv_file='train_split.csv',
        root_dir='/kaggle/input/ai-vs-human-generated-dataset/',
        transform=train_transforms
    )

    val_dataset = AIVSHumanDataset(
        csv_file='val_split.csv',
        root_dir='/kaggle/input/ai-vs-human-generated-dataset/',
        transform=val_transforms
    )

    # Create dataloaders
    batch_size = 16
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

    # Model setup
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = GANDetectorNet().to(device)

    # Use DataParallel if multiple GPUs are available
    if torch.cuda.device_count() > 1:
        print(f"Using {torch.cuda.device_count()} GPUs")
        model = nn.DataParallel(model)

    # Training setup
    criterion = nn.CrossEntropyLoss(label_smoothing=0.15)
    optimizer = optim.AdamW(model.parameters(), lr=1e-3, weight_decay=0.02)
    num_epochs = 20
    
    scheduler = OneCycleLR(
        optimizer,
        max_lr=1e-3,
        epochs=num_epochs,
        steps_per_epoch=len(train_loader),
        pct_start=0.2,
        div_factor=25
    )

    # Training loop
    best_val_acc = 0.0
    for epoch in range(num_epochs):
        train_loss, train_acc = train_one_epoch(model, criterion, optimizer, scheduler, train_loader, device)
        val_loss, val_acc = validate_one_epoch(model, criterion, val_loader, device)
        
        print(f"Epoch [{epoch+1}/{num_epochs}] "
              f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f} "
              f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")
        
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            # When using DataParallel, save the model's module (original model)
            torch.save(model.module.state_dict() if hasattr(model, 'module') else model.state_dict(), 'best_model.pth')

    # Generate predictions
    test_df = pd.read_csv('/kaggle/input/ai-vs-human-generated-dataset/test.csv')
    test_dataset = AIVSHumanTestDataset(
        df=test_df,
        root_dir='/kaggle/input/ai-vs-human-generated-dataset/',
        transform=val_transforms
    )
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

    # Load best model
    # Adjust model loading to work with and without DataParallel
    if torch.cuda.device_count() > 1:
        state_dict = torch.load('best_model.pth')
        model = GANDetectorNet().to(device)
        model = nn.DataParallel(model)
        model.load_state_dict(state_dict)
    else:
        model.load_state_dict(torch.load('best_model.pth'))
    
    model.eval()

    # Generate predictions
    all_preds = []
    with torch.no_grad():
        for images in test_loader:
            images = images.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            all_preds.extend(predicted.cpu().numpy().tolist())

    # Create submission file
    submission_df = pd.DataFrame({
        'id': test_df['id'],
        'label': all_preds
    })
    submission_df.to_csv('submission.csv', index=False)
    print("Submission file saved as submission.csv!")

if __name__ == "__main__":
    main()

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B3_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/efficientnet_b3_rwightman-b3899882.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b3_rwightman-b3899882.pth
100%|██████████| 47.2M/47.2M [00:00<00:00, 168MB/s]


Using 2 GPUs


<ipython-input-1-17dff2ef7bf9>:105: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
